In [1]:
!pip -q install transformers accelerate bitsandbytes sentencepiece
!pip -q install huggingface_hub

In [2]:
from transformers import pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration
import random
import numpy as np

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained(
    "google/flan-t5-large", 
    device_map="auto", 
    load_in_8bit=True, 
    low_cpu_mem_usage=True
)
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-21ebae4wf4f34 --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=

In [3]:
def flan_t5(func):
    def wrapper(prompt):
        to_model = func(prompt)
        completion = generator(to_model)
        return completion[0]['generated_text']
    return wrapper

categories = [
    "Science", "Technology", "Art", "Engineering", "Mathematics"
]

# We're going to generate the few-shot prompt with the T5 model first so we
# don't have to do any of that pesky "prompting" stuff
@flan_t5
def create_example(category):
    return "Create a creative example subject that falls under the given category. " + \
           "For example 'Transportation': 'buses go rogue', 'Soda Cans': 'mt. dew cans have been exploding!'. " + \
           f"Category: {category}"

examples = [create_example(i) for i in categories]
examples = list(zip(examples, categories))
random.shuffle(examples)
examples_prompt = "\n".join([f"{i}: {j}" for i,j in examples])

@flan_t5
def clean_label(text):
    return "Your task is to classify a given label into the following categories: " + \
           f"{', '.join(categories)}." + \
           f"For example: {examples_prompt}" + \
           f"Label: {text}"

In [4]:
print(clean_label("bridge building"))

'Engineering'

In [11]:
dirty_labels = ["bridge building", "alcoholic", "pretzel sculptor", "watermelon portrait creator", "wifi"]
clean_labels = [clean_label(x) for x in dirty_labels]
print(clean_labels)

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


['Engineering', 'Science', 'Science', 'Art', 'Technology']


In [5]:
# I saw another example that used fuzzy scores / probabilities from ChatGPT so here's a way to do that
@flan_t5
def class_confidence_score(data): # {text: 'text', class: 'class'}
    return f"Classes to choose from are: {', '.join(categories)}. " + \
            "Your task is to provide a score from 1-10 for how well a dirty label fits a class name. " + \
            "How well does 'addition' fit the 'College Algebra' class? 5 \n" + \
            "How well does 'space travel' fit the 'Space' class? 10 \n" + \
            "How well does 'tomato soup' fit the 'Fruits' class? 6 \n" + \
            f"How well does '{data['text']}' fit the '{data['class']}' class?"

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def probabilities(x, categories):
    scores = {}
    for i in categories:
        scores[i] = class_confidence_score({'text': x, 'class': i})
    softmaxed_scores = softmax(np.array([int(i) for i in scores.values()])).tolist()
    return softmaxed_scores

probabilities('bridge building', categories)

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[0.017065385160357122,
 0.017065385160357122,
 0.017065385160357122,
 0.9317384593585716,
 0.017065385160357122]